In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

In [2]:
import pandas as pd
import backtrader as bt

In [23]:
class SMCross(bt.Strategy):
    params = dict(
        pfast = 10,
        pslow = 30
    )

    def __init__(self):

        self.close_price = self.datas[0].close
        self.order = None
        fast_sma = bt.ind.SMA(self.datas[0], period=self.p.pfast)
        slow_sma = bt.ind.SMA(self.datas[0], period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(fast_sma, slow_sma)


    def log(self, text, dt=None):
        date = dt or self.datas[0].datetime.date(0)
        print(f'{date}: {text}')


    def notify_trade(self, trade):
        pass

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'Buy has been executed at {order.executed.price}')

            elif order.issell():
                self.log(f'Sell has been executed at {order.executed.price}')
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log("Order has not gone through")
        
        self.order = None

    def next(self):

        if self.order:
            return
            
        if not self.position:
            if self.crossover > 0:
                self.log(f'Buy order at: {self.close_price[1]}')
                self.buy()
                
        elif self.crossover < 0:
            self.log(f'Sell order at: {self.close_price[1]}')
            self.sell()

In [25]:
import datetime
cerebro = bt.Cerebro()
datapath = '../data/TSLA.csv'
data = bt.feeds.YahooFinanceCSVData(dataname=datapath, fromdate=datetime.datetime(2022, 10, 31), todate=datetime.datetime(2023, 10, 30),)  
cerebro.adddata(data)
cerebro.addstrategy(SMCross)
print(f'Opening bal was {cerebro.broker.get_cash()}')
cerebro.run()
print(f'Closing bal is {cerebro.broker.get_cash()}')
#cerebro.plot() 

Opening bal was 10000.0
2023-01-25: Buy order at: 160.27
2023-01-26: Buy has been executed at 159.97
2023-03-10: Sell order at: 174.48
2023-03-13: Sell has been executed at 167.46
2023-03-31: Buy order at: 194.77
2023-04-03: Buy has been executed at 199.91
2023-04-17: Sell order at: 184.31
2023-04-18: Sell has been executed at 187.15
2023-05-19: Buy order at: 188.87
2023-05-22: Buy has been executed at 180.7
2023-08-01: Sell order at: 254.11
2023-08-02: Sell has been executed at 255.57
2023-09-06: Buy order at: 251.49
2023-09-07: Buy has been executed at 245.07
2023-10-02: Sell order at: 246.53
2023-10-03: Sell has been executed at 248.61
2023-10-12: Buy order at: 251.12
2023-10-13: Buy has been executed at 258.9
2023-10-18: Sell order at: 220.11
2023-10-19: Sell has been executed at 225.95
Closing bal is 10040.19
